### Define Loss Functions

In [13]:
"""Metrics to assess performance on ordinal classification task given class prediction
   using hyper plane loss techniques 
"""

# Authors: Bob Vanderheyden <rvanderh@us.ibm.com>
#          Ying Xie <yxie2@kennesaw.edu>
#         
# Contributor: Shayan Shamskolahi

import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
import numpy as np

def hpall_mean_loss(y_true, y_pred, minlabel, maxlabel, margin=0.1, ordering_loss_weight=1):
    """ Evaluate the ordinal hyperplane ordering loss and point loss of the predictions y_pred\
        (using reduce mean).

        Parameters
        ----------
        y_true : array-like
        y_pred : array-like
        minlabel : integer
        maxlabel : integer
        margin : float
        ordering_loss_weight : float

        Returns
        -------
        loss: float
        A non-negative floating point value (best value is 0.0)
        
        Usage
        -------
        loss = hp_all_loss([4,1,2,0,4,2,1], [6.0,3.1,5.2,1.0,4.0,2.2,3.7],0,4,.3,0.1)
        print('Loss: ', loss.numpy()) # Loss: 0.7228571
        
        
        Usage with the `compile` API:
        
        ```python
        
        Example Keras wrapper for hp_all_loss:
        
        def get_ohpl_wrapper (min_label, max_label, margin, ordering_loss_weight):
            def ohpl(y_true, y_pred):
                return hpall_mean_loss(y_true, y_pred, min_label, max_label, margin, ordering_loss_weight)
            return ohpl

        loss = get_ohpl_wrapper(2,7,.3,1) # ordering_loss_weight must not be less that 1
        
        model = tf.keras.Model(inputs, outputs)
        model.compile(loss=hp_all_loss, optimizer='adam', loss=ohpl_point_loss)
        ```
        
    """
    
    min_label = tf.constant(minlabel, dtype=tf.float32)
    max_label = tf.constant(maxlabel, dtype=tf.float32)
    margin = tf.constant(margin, dtype=tf.float32) # centroid margin
    ordering_loss_weight = tf.constant(ordering_loss_weight, dtype=tf.float32) 
    
    y_pred = tf.convert_to_tensor(y_pred)
    y_true = tf.dtypes.cast(y_true, y_pred.dtype)
    y_pred = tf.reshape(tf.transpose(y_pred),[-1,1])
    
    # OHPL ordering loss
    # one hot vector for y_true
    ords, idx = tf.unique(tf.reshape(y_true, [-1])) 
    num = tf.shape(ords)[0]
    y_true_1hot = tf.one_hot(idx, num)

    # mean distance for each class
    yO = tf.matmul(tf.transpose(y_pred),y_true_1hot)
    yc = tf.reduce_sum(y_true_1hot,0)
    class_mean = tf.divide(yO,yc)  

    # min. distance
    ords = tf.dtypes.cast(ords, tf.float32)
    ords0 = tf.reshape(ords, [-1,1])
    ords1 = tf.reshape(ords, [1,-1])
    
    min_distance = tf.subtract(ords0, ords1)
    # apply ReLU
    min_distance = tf.nn.relu (min_distance)
    
    # keeps min. distance
    keep = tf.minimum(min_distance,1)

    # distance to centroid     
    class_mean0 = tf.reshape(class_mean, [-1,1])
    class_mean1 = tf.reshape(class_mean, [1,-1])
    class_mean = tf.subtract(class_mean0, class_mean1)  
    # apply ReLU    
    class_mean = tf.nn.relu(class_mean)
    centroid_distance = tf.multiply(keep, class_mean)
    
    hp_ordering_loss = tf.subtract(min_distance,centroid_distance)
    # apply ReLU
    hp_ordering_loss = tf.nn.relu(hp_ordering_loss)
    hp_ordering_loss = tf.reduce_sum(hp_ordering_loss)
    
    # OHPL point loss
    # Centroid for point
    point_cent = tf.matmul(y_true_1hot, class_mean0)
    
    lower_bound = tf.subtract(min_label,y_true)
    lower_bound = tf.add(lower_bound,1)
    lower_bound = tf.multiply(lower_bound,1e9)
    # apply ReLU    
    lower_bound = tf.nn.relu(lower_bound)
    lower_bound = tf.add(margin, lower_bound)

    upper_bound = tf.subtract(y_true,max_label)
    upper_bound = tf.add(upper_bound,1)
    upper_bound = tf.multiply(upper_bound,1e9)
    # apply ReLU    
    upper_bound = tf.nn.relu(upper_bound)
    upper_bound = tf.add(margin, upper_bound)    

    upper_loss = tf.add(point_cent,upper_bound[:,tf.newaxis])
    upper_loss = tf.subtract(y_pred,upper_loss)
    # apply ReLU    
    upper_loss = tf.nn.relu(upper_loss)
    
    lower_loss = tf.add(lower_bound[:,tf.newaxis],y_pred)
    lower_loss = tf.subtract(point_cent,lower_loss)
    # apply ReLU    
    lower_loss = tf.nn.relu(lower_loss)
   
    hp_point_loss = tf.add(upper_loss, lower_loss)
    hp_point_loss = tf.reduce_mean(hp_point_loss)

    # aggregate ordering loss and point loss     
    mean_loss = tf.add(hp_point_loss,tf.multiply(ordering_loss_weight, hp_ordering_loss))
    
    return mean_loss

   
    """    
        References
        ----------
        .. [1] Vanderheyden, Bob and Ying Xie. Ordinal Hyperplane Loss. (2018). 
           2018 IEEE International Conference on Big Data (Big Data), 
           2018 IEEE International Conference On, 2337. https://doi-org.proxy.kennesaw.edu/10.1109/BigData.2018.8622079
    """

In [14]:
def hpall_sum_loss(y_true, y_pred, minlabel, maxlabel, margin=0.1, ordering_loss_weight=1):
    """ Evaluate the ordinal hyperplane ordering loss and point loss of the predictions y_pred\
        (using reduce sum).

        Parameters
        ----------
        y_true : array-like
        y_pred : array-like
        minlabel : integer
        maxlabel : integer
        margin : float
        ordering_loss_weight : float

        Returns
        -------
        loss: float
        A non-negative floating point value (best value is 0.0)
        
        Usage
        -------
        loss = hp_all_loss([4,1,2,0,4,2,1], [6.0,3.1,5.2,1.0,4.0,2.2,3.7],0,4,.3,0.1)
        print('Loss: ', loss.numpy()) # Loss: 3.48
        
        
        Usage with the `compile` API:
        
        ```python
        
        Example Keras wrapper for hp_all_loss:
        
        def get_ohpl_wrapper (min_label, max_label, margin, ordering_loss_weight):
            def ohpl(y_true, y_pred):
                return hpall_sum_loss(y_true, y_pred, min_label, max_label, margin, ordering_loss_weight)
            return ohpl

        loss = get_ohpl_wrapper(0,4,1,1)
        
        model = tf.keras.Model(inputs, outputs)
        model.compile(loss=hp_all_loss, optimizer='adam', loss=ohpl_point_loss)
        ```
        
    """
    
    min_label = tf.constant(minlabel, dtype=tf.float32)
    max_label = tf.constant(maxlabel, dtype=tf.float32)
    margin = tf.constant(margin, dtype=tf.float32) # centroid margin
    ordering_loss_weight = tf.constant(ordering_loss_weight, dtype=tf.float32) 
    
    y_pred = tf.convert_to_tensor(y_pred)
    y_true = tf.dtypes.cast(y_true, y_pred.dtype)
    y_pred = tf.reshape(tf.transpose(y_pred),[-1,1])
    
    # OHPL ordering loss
    # one hot vector for y_true
    ords, idx = tf.unique(tf.reshape(y_true, [-1])) 
    num = tf.shape(ords)[0]
    y_true_1hot = tf.one_hot(idx, num)

    # mean distance for each class
    yO = tf.matmul(tf.transpose(y_pred),y_true_1hot)
    yc = tf.reduce_sum(y_true_1hot,0)
    class_mean = tf.divide(yO,yc)  

    # min. distance
    ords = tf.dtypes.cast(ords, tf.float32)
    ords0 = tf.reshape(ords, [-1,1])
    ords1 = tf.reshape(ords, [1,-1])
    
    min_distance = tf.subtract(ords0, ords1)
    # apply ReLU
    min_distance = tf.nn.relu (min_distance)
    
    # keeps min. distance
    keep = tf.minimum(min_distance,1)

    # distance to centroid     
    class_mean0 = tf.reshape(class_mean, [-1,1])
    class_mean1 = tf.reshape(class_mean, [1,-1])
    class_mean = tf.subtract(class_mean0, class_mean1)  
    # apply ReLU    
    class_mean = tf.nn.relu(class_mean)
    centroid_distance = tf.multiply(keep, class_mean)
    
    hp_ordering_loss = tf.subtract(min_distance,centroid_distance)
    # apply ReLU
    hp_ordering_loss = tf.nn.relu(hp_ordering_loss)
    hp_ordering_loss = tf.reduce_sum(hp_ordering_loss)
    
    # OHPL point loss
    # Centroid for point
    point_cent = tf.matmul(y_true_1hot, class_mean0)
    
    lower_bound = tf.subtract(min_label,y_true)
    lower_bound = tf.add(lower_bound,1)
    lower_bound = tf.multiply(lower_bound,1e9)
    # apply ReLU    
    lower_bound = tf.nn.relu(lower_bound)
    lower_bound = tf.add(margin, lower_bound)

    upper_bound = tf.subtract(y_true,max_label)
    upper_bound = tf.add(upper_bound,1)
    upper_bound = tf.multiply(upper_bound,1e9)
    # apply ReLU    
    upper_bound = tf.nn.relu(upper_bound)
    upper_bound = tf.add(margin, upper_bound)    

    upper_loss = tf.add(point_cent,upper_bound[:,tf.newaxis])
    upper_loss = tf.subtract(y_pred,upper_loss)
    # apply ReLU    
    upper_loss = tf.nn.relu(upper_loss)
    
    lower_loss = tf.add(lower_bound[:,tf.newaxis],y_pred)
    lower_loss = tf.subtract(point_cent,lower_loss)
    # apply ReLU    
    lower_loss = tf.nn.relu(lower_loss)
   
    hp_point_loss = tf.add(upper_loss, lower_loss)
    hp_point_loss = tf.reduce_sum(hp_point_loss)

    # aggregate ordering loss and point loss     
    sum_loss = tf.add(hp_point_loss,tf.multiply(ordering_loss_weight, hp_ordering_loss))
    
    return sum_loss


    """    
        References
        ----------
        .. [1] Vanderheyden, Bob and Ying Xie. Ordinal Hyperplane Loss. (2018). 
           2018 IEEE International Conference on Big Data (Big Data), 
           2018 IEEE International Conference On, 2337. https://doi-org.proxy.kennesaw.edu/10.1109/BigData.2018.8622079
    """

### Test the result:

In [3]:
loss = hpall_mean_loss([4,1,2,0,4,2,1], [6.0,3.1,5.2,1.0,4.0,2.2,3.7],0,4,.3,0.1)
print('Loss: ', loss.numpy()) # Loss: 0.7228571

Loss:  0.7228571


In [4]:
loss = hpall_sum_loss([4,1,2,0,4,2,1], [6.0,3.1,5.2,1.0,4.0,2.2,3.7],0,4,.3,0.1)
print('Loss: ', loss.numpy()) # Loss: 3.48

Loss:  3.48


### Application in Keras (mean loss):

In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [2]:
# prepare the data
pwd = !pwd
df = pd.read_csv('world_happiness_2015_2019.csv')
df.Score = df.Score.astype('int32')
df.drop(['Year'], axis=1, inplace=True)
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 781 entries, 0 to 781
Data columns (total 7 columns):
Score                           781 non-null int32
GDP per capita                  781 non-null float64
Social support                  781 non-null float64
Healthy life expectancy         781 non-null float64
Freedom to make life choices    781 non-null float64
Generosity                      781 non-null float64
Perceptions of corruption       781 non-null float64
dtypes: float64(6), int32(1)
memory usage: 45.8 KB


In [9]:
X = df.iloc[:,1:]/df.iloc[:,1:].max()
y = df.iloc[:,0]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [12]:
print(X_train.max(), X_train.min())
print(np.unique(y, return_counts=True))

GDP per capita                  1.000000
Social support                  1.000000
Healthy life expectancy         0.983348
Freedom to make life choices    0.943370
Generosity                      1.000000
Perceptions of corruption       1.000000
dtype: float64 GDP per capita                  0.0
Social support                  0.0
Healthy life expectancy         0.0
Freedom to make life choices    0.0
Generosity                      0.0
Perceptions of corruption       0.0
dtype: float64
(array([2, 3, 4, 5, 6, 7], dtype=int32), array([  7,  89, 202, 249, 162,  72]))


### Wrapper in action - Keras sequential model:

In [15]:
# example Keras wrapper for hpall_mean_loss

def get_ohpl_wrapper (min_label, max_label, margin, ordering_loss_weight):
    def ohpl(y_true, y_pred):
        return hpall_mean_loss(y_true, y_pred, min_label, max_label, margin, ordering_loss_weight)
    return ohpl

loss = get_ohpl_wrapper(2,7,.1,1) # ordering_loss_weight must not be less that 1

In [ ]:
# Determine Hyperparameter for Ordering Loss Weight
# Code frequently (every 5 epochs) calculates the centroids and calculates the minimum and maximum gap (distance between adjcent centroids)
# If the algorithm requires more than 50 epochs to establish the ordering and spacing, increase the ordering_loss_weight value above. 

In [57]:
from tensorflow.keras.optimizers import Adam

y_train = np.array(y_train)
labels, freq = np.unique(y_train, return_counts=True)
ords = labels.reshape(-1, 1)
labs = len(labels)

eyes = np.eye(labs)
cDiff = eyes[1:labs,:]-eyes[:(labs-1),:]

# Create matrix from on hot encoded training labels to use to calculate class centroids
onehot_encoder = OneHotEncoder(sparse=False, categories='auto')
onehot = onehot_encoder.fit_transform(y_train.reshape((-1, 1)))
onehot_inverse = 1/np.sum((onehot.T), axis=1)
new_y_train = onehot.T*onehot_inverse.reshape(-1,1)

model = Sequential()
model.add(Dense(25, activation='relu', input_shape=(6, )))
model.add(Dropout(0.1))
model.add(Dense(30, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(6, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1))

opt = Adam(lr=0.003, decay=1e-5)
model.compile(loss=loss, optimizer=opt)

mingap = 0.0
while mingap < 1.0:
    model.fit(X_train, y_train, epochs=5, batch_size=16, shuffle=True, verbose = 0)

    # Score the training set
    pred = model.predict(X_train, batch_size=5)
    train_cent = np.matmul(new_y_train, pred)
    mingap = min(np.matmul(cDiff, train_cent))[0]
    maxgap = max(np.matmul(cDiff, train_cent))[0]
    print('Centers', mingap, maxgap)
    print(train_cent.T)
    print(' ')


Centers 0.9388336274272973 2.8744346328730974
[[-9.76780319 -7.63711633 -5.74853849 -2.87410386 -1.07495029 -0.13611666]]
 
Centers 1.4078685059388119 3.075345569764254
[[-10.94556073  -8.35737618  -6.26044987  -3.1851043   -0.97623344
    0.43163507]]
 


In [46]:
# Define and compile the model 
from tensorflow.keras.optimizers import Adam

model = Sequential()
model.add(Dense(25, activation='relu', input_shape=(6, )))
model.add(Dropout(0.1))
model.add(Dense(30, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(6, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1))

opt = Adam(lr=0.003, decay=1e-5)
model.compile(loss=loss, optimizer=opt)


model.fit(X_train, y_train, epochs=2000, batch_size=16, shuffle=True)

Epoch 1/2000
523/523 [==============================] - 1s 2ms/sample - loss: 19.3022
Epoch 2/2000
523/523 [==============================] - 0s 153us/sample - loss: 13.7326
Epoch 3/2000
523/523 [==============================] - 0s 154us/sample - loss: 2.9801
Epoch 4/2000
523/523 [==============================] - 0s 138us/sample - loss: 2.3599
Epoch 5/2000
523/523 [==============================] - 0s 150us/sample - loss: 2.3687
Epoch 6/2000
523/523 [==============================] - 0s 140us/sample - loss: 1.8646
Epoch 7/2000
523/523 [==============================] - 0s 148us/sample - loss: 1.3638
Epoch 8/2000
523/523 [==============================] - 0s 157us/sample - loss: 1.7370
Epoch 9/2000
523/523 [==============================] - 0s 153us/sample - loss: 1.3120
Epoch 10/2000
523/523 [==============================] - 0s 162us/sample - loss: 1.3064
Epoch 11/2000
523/523 [==============================] - 0s 142us/sample - loss: 1.7613
Epoch 12/2000
523/523 [==================

523/523 [==============================] - 0s 146us/sample - loss: 0.8550
Epoch 187/2000
523/523 [==============================] - 0s 142us/sample - loss: 0.8908
Epoch 188/2000
523/523 [==============================] - 0s 139us/sample - loss: 0.9748
Epoch 189/2000
523/523 [==============================] - 0s 140us/sample - loss: 0.8959
Epoch 190/2000
523/523 [==============================] - 0s 151us/sample - loss: 0.8624
Epoch 191/2000
523/523 [==============================] - 0s 143us/sample - loss: 0.7717
Epoch 192/2000
523/523 [==============================] - 0s 132us/sample - loss: 0.8031
Epoch 193/2000
523/523 [==============================] - 0s 146us/sample - loss: 0.8883
Epoch 194/2000
523/523 [==============================] - 0s 156us/sample - loss: 0.9275
Epoch 195/2000
523/523 [==============================] - 0s 154us/sample - loss: 0.7190
Epoch 196/2000
523/523 [==============================] - 0s 144us/sample - loss: 0.7291
Epoch 197/2000
523/523 [============

523/523 [==============================] - 0s 136us/sample - loss: 0.6680
Epoch 371/2000
523/523 [==============================] - 0s 131us/sample - loss: 0.8323
Epoch 372/2000
523/523 [==============================] - 0s 130us/sample - loss: 0.5870
Epoch 373/2000
523/523 [==============================] - 0s 136us/sample - loss: 0.5813
Epoch 374/2000
523/523 [==============================] - 0s 129us/sample - loss: 0.7611
Epoch 375/2000
523/523 [==============================] - 0s 129us/sample - loss: 0.9364
Epoch 376/2000
523/523 [==============================] - 0s 129us/sample - loss: 0.7635
Epoch 377/2000
523/523 [==============================] - 0s 129us/sample - loss: 0.7687
Epoch 378/2000
523/523 [==============================] - 0s 130us/sample - loss: 0.6660
Epoch 379/2000
523/523 [==============================] - 0s 134us/sample - loss: 0.7768
Epoch 380/2000
523/523 [==============================] - 0s 129us/sample - loss: 0.7417
Epoch 381/2000
523/523 [============

523/523 [==============================] - 0s 135us/sample - loss: 0.5436
Epoch 555/2000
523/523 [==============================] - 0s 138us/sample - loss: 0.9248
Epoch 556/2000
523/523 [==============================] - 0s 135us/sample - loss: 0.7015
Epoch 557/2000
523/523 [==============================] - 0s 139us/sample - loss: 0.7574
Epoch 558/2000
523/523 [==============================] - 0s 138us/sample - loss: 0.6564
Epoch 559/2000
523/523 [==============================] - 0s 139us/sample - loss: 0.6149
Epoch 560/2000
523/523 [==============================] - 0s 147us/sample - loss: 0.8213
Epoch 561/2000
523/523 [==============================] - 0s 137us/sample - loss: 0.7727
Epoch 562/2000
523/523 [==============================] - 0s 137us/sample - loss: 0.7649
Epoch 563/2000
523/523 [==============================] - 0s 137us/sample - loss: 0.6969
Epoch 564/2000
523/523 [==============================] - 0s 142us/sample - loss: 0.7261
Epoch 565/2000
523/523 [============

523/523 [==============================] - 0s 138us/sample - loss: 0.4852
Epoch 739/2000
523/523 [==============================] - 0s 139us/sample - loss: 0.4923
Epoch 740/2000
523/523 [==============================] - 0s 138us/sample - loss: 0.5277
Epoch 741/2000
523/523 [==============================] - 0s 136us/sample - loss: 0.8255
Epoch 742/2000
523/523 [==============================] - 0s 138us/sample - loss: 0.6314
Epoch 743/2000
523/523 [==============================] - 0s 136us/sample - loss: 0.8106
Epoch 744/2000
523/523 [==============================] - 0s 137us/sample - loss: 0.8157
Epoch 745/2000
523/523 [==============================] - 0s 137us/sample - loss: 0.5316
Epoch 746/2000
523/523 [==============================] - 0s 140us/sample - loss: 0.5755
Epoch 747/2000
523/523 [==============================] - 0s 138us/sample - loss: 0.5352
Epoch 748/2000
523/523 [==============================] - 0s 138us/sample - loss: 0.4620
Epoch 749/2000
523/523 [============

...

523/523 [==============================] - 0s 190us/sample - loss: 0.5829
Epoch 1197/2000
523/523 [==============================] - 0s 198us/sample - loss: 0.4753
Epoch 1198/2000
523/523 [==============================] - 0s 190us/sample - loss: 0.6520
Epoch 1199/2000
523/523 [==============================] - 0s 209us/sample - loss: 0.8719
Epoch 1200/2000
523/523 [==============================] - 0s 191us/sample - loss: 0.6960
Epoch 1201/2000
523/523 [==============================] - 0s 194us/sample - loss: 0.7073
Epoch 1202/2000
523/523 [==============================] - 0s 188us/sample - loss: 0.5583
Epoch 1203/2000
523/523 [==============================] - 0s 188us/sample - loss: 0.5134
Epoch 1204/2000
523/523 [==============================] - 0s 190us/sample - loss: 0.6721
Epoch 1205/2000
523/523 [==============================] - 0s 193us/sample - loss: 0.5873
Epoch 1206/2000
523/523 [==============================] - 0s 190us/sample - loss: 0.6007
Epoch 1207/2000
523/523 [=

523/523 [==============================] - 0s 168us/sample - loss: 0.5662
Epoch 1379/2000
523/523 [==============================] - 0s 167us/sample - loss: 0.4567
Epoch 1380/2000
523/523 [==============================] - 0s 170us/sample - loss: 0.3360
Epoch 1381/2000
523/523 [==============================] - 0s 171us/sample - loss: 0.3531
Epoch 1382/2000
523/523 [==============================] - 0s 170us/sample - loss: 0.5770
Epoch 1383/2000
523/523 [==============================] - 0s 172us/sample - loss: 0.4649
Epoch 1384/2000
523/523 [==============================] - 0s 176us/sample - loss: 0.5087
Epoch 1385/2000
523/523 [==============================] - 0s 169us/sample - loss: 0.5632
Epoch 1386/2000
523/523 [==============================] - 0s 172us/sample - loss: 0.5387
Epoch 1387/2000
523/523 [==============================] - 0s 169us/sample - loss: 0.6474
Epoch 1388/2000
523/523 [==============================] - 0s 170us/sample - loss: 0.4223
Epoch 1389/2000
523/523 [=

523/523 [==============================] - 0s 166us/sample - loss: 0.5699
Epoch 1561/2000
523/523 [==============================] - 0s 165us/sample - loss: 0.6733
Epoch 1562/2000
523/523 [==============================] - 0s 164us/sample - loss: 0.6549
Epoch 1563/2000
523/523 [==============================] - 0s 161us/sample - loss: 0.5321
Epoch 1564/2000
523/523 [==============================] - 0s 172us/sample - loss: 0.5447
Epoch 1565/2000
523/523 [==============================] - 0s 166us/sample - loss: 0.5773
Epoch 1566/2000
523/523 [==============================] - 0s 164us/sample - loss: 0.4263
Epoch 1567/2000
523/523 [==============================] - 0s 163us/sample - loss: 0.5243
Epoch 1568/2000
523/523 [==============================] - 0s 167us/sample - loss: 0.5976
Epoch 1569/2000
523/523 [==============================] - 0s 164us/sample - loss: 0.4786
Epoch 1570/2000
523/523 [==============================] - 0s 165us/sample - loss: 0.5203
Epoch 1571/2000
523/523 [=

523/523 [==============================] - 0s 182us/sample - loss: 0.5421
Epoch 1743/2000
523/523 [==============================] - 0s 185us/sample - loss: 0.5383
Epoch 1744/2000
523/523 [==============================] - 0s 190us/sample - loss: 0.4686
Epoch 1745/2000
523/523 [==============================] - 0s 183us/sample - loss: 0.4467
Epoch 1746/2000
523/523 [==============================] - 0s 186us/sample - loss: 0.4549
Epoch 1747/2000
523/523 [==============================] - 0s 184us/sample - loss: 0.6107
Epoch 1748/2000
523/523 [==============================] - 0s 183us/sample - loss: 0.4886
Epoch 1749/2000
523/523 [==============================] - 0s 188us/sample - loss: 0.4492
Epoch 1750/2000
523/523 [==============================] - 0s 185us/sample - loss: 0.3940
Epoch 1751/2000
523/523 [==============================] - 0s 186us/sample - loss: 0.5498
Epoch 1752/2000
523/523 [==============================] - 0s 182us/sample - loss: 0.5411
Epoch 1753/2000
523/523 [=

523/523 [==============================] - 0s 182us/sample - loss: 0.4592
Epoch 1925/2000
523/523 [==============================] - 0s 178us/sample - loss: 0.6050
Epoch 1926/2000
523/523 [==============================] - 0s 178us/sample - loss: 0.4766
Epoch 1927/2000
523/523 [==============================] - 0s 182us/sample - loss: 0.5435
Epoch 1928/2000
523/523 [==============================] - 0s 179us/sample - loss: 0.4588
Epoch 1929/2000
523/523 [==============================] - 0s 181us/sample - loss: 0.6482
Epoch 1930/2000
523/523 [==============================] - 0s 183us/sample - loss: 0.6814
Epoch 1931/2000
523/523 [==============================] - 0s 183us/sample - loss: 0.4561
Epoch 1932/2000
523/523 [==============================] - 0s 191us/sample - loss: 0.8305
Epoch 1933/2000
523/523 [==============================] - 0s 180us/sample - loss: 0.6160
Epoch 1934/2000
523/523 [==============================] - 0s 207us/sample - loss: 0.6926
Epoch 1935/2000
523/523 [=

In [47]:
# Define the minimum class
min_class = min(np.unique(y_train))


In [50]:
# Multiply centroid calculation matrix, new_y_train, by training set scores
train_cent = np.matmul(new_y_train, pred)
train_cent

array([[-8.53362376],
       [-3.62544511],
       [-1.27808337],
       [ 0.70373358],
       [ 2.55631305],
       [ 4.84041836]])

In [51]:
# Calculate new data model score
new_pred = model.predict(X_test)

In [52]:
# Identify the closest centroid
rcenter = train_cent.T # create row matrix of centroids
y_pred = np.argmin(abs(new_pred - rcenter), axis=1) + min_class      

In [53]:
# calculate the mean absolute error and mean zero one error
mae = np.mean(abs(y_pred - y_test))
mze = np.mean(abs(y_pred - y_test) > 0)   
print(mae, mze)

0.4573643410852713 0.43023255813953487


In [54]:
# Confusion matrix
from sklearn.metrics import confusion_matrix 
confusion_matrix(y_test, y_pred) 

array([[ 0,  0,  0,  0,  0,  0],
       [ 1, 19,  7,  0,  0,  0],
       [ 1, 10, 37, 16,  1,  0],
       [ 0,  1, 11, 43, 28,  2],
       [ 0,  0,  1, 10, 30, 14],
       [ 0,  0,  0,  1,  7, 18]])